In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


https://keras.io/examples/nlp/text_classification_with_transformer/

In [44]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense, Embedding, LSTM, Bidirectional
from sklearn.model_selection import train_test_split

In [45]:
pwd_train_40 = '/content/drive/MyDrive/Colab Notebooks/NLP proj/data_thresh40/train_data.csv'
pwd_val_40 = '/content/drive/MyDrive/Colab Notebooks/NLP proj/data_thresh40/val_data.csv'
pwd_test_40 = '/content/drive/MyDrive/Colab Notebooks/NLP proj/data_thresh40/test_data.csv'

train_df = pd.read_csv(pwd_train_40)
val_df = pd.read_csv(pwd_val_40)
test_df = pd.read_csv(pwd_test_40)

In [46]:
X_train = train_df['clean_content']
X_val = val_df['clean_content']
X_test = test_df['clean_content']

In [87]:
vocab_size = 3000
max_length = 300
oov_token = "<UNK>"
padding_type = "post"
trunction_type='post'
tokenizer = Tokenizer(num_words = vocab_size,oov_token=oov_token,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r')
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index


In [75]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunction_type)
X_val_sequences = tokenizer.texts_to_sequences(X_val)
X_val_padded = pad_sequences(X_val_sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunction_type)

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunction_type)


In [49]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_df['author'])
y_train=le.transform(train_df['author'])
y_val=le.transform(val_df['author'])

In [10]:
y_test=le.transform(test_df['author'])

In [50]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output


In [51]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [52]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


In [23]:
set(y_train)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

In [76]:
embed_dim = 32  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
nclasses = 12

inputs = layers.Input(shape=(max_length,))
embedding_layer = TokenAndPositionEmbedding(max_length, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(nclasses, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [77]:
#lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#    initial_learning_rate=1e-2,
#    decay_steps=10000,
#    decay_rate=0.9)
#opt = keras.optimizers.Adam(learning_rate=lr_schedule)
#model.compile(opt, "sparse_categorical_crossentropy", metrics=["accuracy"])

model.compile('adam', "sparse_categorical_crossentropy", metrics=["accuracy"])


In [78]:
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=40, validation_data=(X_val_padded, y_val)
)

Epoch 1/40
15/15 [==============================] - 3s 219ms/step - loss: 2.4888 - accuracy: 0.0919 - val_loss: 2.4416 - val_accuracy: 0.1024
Epoch 2/40
15/15 [==============================] - 3s 202ms/step - loss: 2.4342 - accuracy: 0.1050 - val_loss: 2.4209 - val_accuracy: 0.1024
Epoch 3/40
15/15 [==============================] - 3s 202ms/step - loss: 2.3849 - accuracy: 0.1204 - val_loss: 2.4107 - val_accuracy: 0.1260
Epoch 4/40
15/15 [==============================] - 3s 220ms/step - loss: 2.3513 - accuracy: 0.1291 - val_loss: 2.3720 - val_accuracy: 0.1102
Epoch 5/40
15/15 [==============================] - 3s 202ms/step - loss: 2.2943 - accuracy: 0.1904 - val_loss: 2.3269 - val_accuracy: 0.1496
Epoch 6/40
15/15 [==============================] - 3s 202ms/step - loss: 2.2350 - accuracy: 0.1904 - val_loss: 2.2773 - val_accuracy: 0.2126
Epoch 7/40
15/15 [==============================] - 3s 207ms/step - loss: 2.1567 - accuracy: 0.2473 - val_loss: 2.2200 - val_accuracy: 0.2756
Epoch 

### with glove embedding
not working lol

In [79]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip


--2020-11-23 20:01:16--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-11-23 20:01:16--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-11-23 20:01:16--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [80]:
! ls

drive		   glove.6B.200d.txt  glove.6B.50d.txt	sample_data
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip


In [81]:
import os
import pathlib


In [108]:
embeddings_index = {}
with open('/content/glove.6B.100d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))


Found 400000 word vectors.


In [109]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))

for word, i in word_index.items():

    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [110]:
glove_embedding_layer = Embedding(len(word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

<tf.Tensor 'embedding_25/embedding_lookup_1/Identity_1:0' shape=(None, 20, 300) dtype=float32>

In [113]:
from keras.layers import Flatten

In [118]:
embed_dim = 32  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
nclasses = 12

inputs = layers.Input(shape=(max_length,))
#print(inputs.shape)
embedding_layer = TokenAndPositionEmbedding(max_length, vocab_size, embed_dim)
x = embedding_layer(inputs)
print(x.shape)
x = glove_embedding_layer(x)
print(x.shape)
#x = Flatten(x)
#print(x.shape)
#transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
transformer_block = TransformerBlock(100, num_heads, 100)
x = transformer_block(x)
print(x.shape)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(nclasses, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


(None, 300, 32)
(None, 300, 32, 100)
(None, 300, 32, 100)


In [119]:
model.compile('adam', "sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(
    X_train_padded, y_train, batch_size=32, epochs=40, validation_data=(X_val_padded, y_val)
)

Epoch 1/40
